In many applications, data may be spread across a number of files or databases or be
arranged in a form that is not easy to analyze. This chapter focuses on tools to help
combine, join, and rearrange data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 8.1 Hierarchical Indexing

Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. 

In [2]:
data = pd.Series(data=np.random.randn(9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd']
                                                 , [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
data

a  1    2.370858
   2    1.693700
   3    0.948044
b  1    0.969922
   3   -0.291930
c  1    0.433798
   2    1.124377
d  2   -0.624081
   3   -0.922000
dtype: float64

What you’re seeing is a prettified view of a Series with a MultiIndex as its index. The
“gaps” in the index display mean “use the label directly above”

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, so-called partial indexing is possible, enabling
you to concisely select subsets of the data

In [5]:
data

a  1    2.370858
   2    1.693700
   3    0.948044
b  1    0.969922
   3   -0.291930
c  1    0.433798
   2    1.124377
d  2   -0.624081
   3   -0.922000
dtype: float64

In [6]:
data['b':'d']

b  1    0.969922
   3   -0.291930
c  1    0.433798
   2    1.124377
d  2   -0.624081
   3   -0.922000
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1    0.969922
   3   -0.291930
d  2   -0.624081
   3   -0.922000
dtype: float64

In [8]:
data.loc[:, 2]

a    1.693700
c    1.124377
d   -0.624081
dtype: float64

In [9]:
data.loc[:,3]

a    0.948044
b   -0.291930
d   -0.922000
dtype: float64

In [10]:
data.loc['b', 3]

-0.2919302515764976

In [11]:
data.unstack() #Convert multi index series into df

,1,2,3
a,2.370858,1.693700,0.948044
b,0.969922,NaN,-0.291930
c,0.433798,1.124377,NaN
d,NaN,-0.624081,-0.922000


In [12]:
data.unstack().stack() #stack reverses the unstack or convert df into multi index series

a  1    2.370858
   2    1.693700
   3    0.948044
b  1    0.969922
   3   -0.291930
c  1    0.433798
   2    1.124377
d  2   -0.624081
   3   -0.922000
dtype: float64

With a DataFrame, either axis can have a hierarchical index

In [13]:
frame = pd.DataFrame(data=np.arange(12).reshape(4,3), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

In [14]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [15]:
frame.index.names = ['key1', 'key2']

In [16]:
frame.columns.names = ['state', 'color']

In [17]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [18]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [19]:
frame['Colorado']

color      Green
key1 key2       
a    1         2
     2         5
b    1         8
     2        11

## Reordering and Sorting Levels

In [20]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [21]:
frame.swaplevel('key1', 'key2') #.swaplevel() swaps the level of indexes

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [22]:
frame.sort_index(level=1) #.sort_index() sorts the index by taking the level argument

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [23]:
frame.swaplevel(1, 0).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

In [24]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [25]:
frame.sum(level='key1')

state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

In [26]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [27]:
frame.sum(level='state', axis=1)

state      Ohio  Colorado
key1 key2                
a    1        1         2
     2        7         5
b    1       13         8
     2       19        11

In [28]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with a DataFrame’s columns

In [29]:
df = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

In [30]:
df

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [31]:
df2 = df.set_index(['c', 'd'])

In [32]:
df2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [33]:
df

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [34]:
df.set_index(['c', 'd'], drop=False) #sets the passed column(s) as index, might hierarchical or not

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [35]:
df2.reset_index() #inverse of set_index(), reset the index to int

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:
1. **pandas.merge** connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.
2. **pandas.concat** concatenates or “stacks” together objects along an axis.
3. The **combine_first** instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

## Database-Style DataFrame Joins

In [36]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

In [37]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [38]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [39]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [40]:
df1.merge(df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [41]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [42]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

In [43]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [44]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [45]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [46]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [47]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer')[['lkey', 'rkey', 'data1', 'data2']]

,lkey,rkey,data1,data2
0,b,b,0.0,1.0
1,b,b,1.0,1.0
2,b,b,6.0,1.0
3,a,a,2.0,0.0
4,a,a,4.0,0.0
5,a,a,5.0,0.0
6,c,NaN,3.0,NaN
7,NaN,d,NaN,2.0


*See Table 8-1. Different join types with how argument*

In [48]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})

In [49]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})

In [50]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [51]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [52]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [53]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'], 'lval': [1, 2, 3]})

In [54]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 'key2': ['one', 'one', 'one', 'two'], 'rval': [4, 5, 6, 7]})

In [55]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [56]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [57]:
pd.merge(right, left, on=['key1', 'key2'])

,key1,key2,rval,lval
0,foo,one,4,1
1,foo,one,5,1
2,bar,one,6,3


In [58]:
pd.merge(right, left, on=['key1', 'key2'], how='outer')

,key1,key2,rval,lval
0,foo,one,4.0,1.0
1,foo,one,5.0,1.0
2,bar,one,6.0,3.0
3,bar,two,7.0,NaN
4,foo,two,NaN,2.0


In [59]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [60]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [61]:
pd.merge(right, left, on=['key1', 'key2'], suffixes=('_right', '_left'), indicator=True, how='outer')

,key1,key2,rval,lval,_merge
0,foo,one,4.0,1.0,both
1,foo,one,5.0,1.0,both
2,bar,one,6.0,3.0,both
3,bar,two,7.0,NaN,left_only
4,foo,two,NaN,2.0,right_only


*See Table 8-2. merge function arguments*

## Merging on Index

In [62]:
left1 = pd.DataFrame({'key1': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})

In [63]:
right1 = pd.DataFrame({'group_vale': [3.5, 7]}, index=['a', 'b'])

In [64]:
left1

,key1,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [65]:
right1

,group_vale
a,3.5
b,7.0


In [66]:
pd.merge(left1, right1, left_on='key1', right_index=True)

,key1,value,group_vale
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [67]:
pd.merge(left1, right1, left_on='key1', right_index=True, how='outer')

,key1,value,group_vale
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [68]:
left_df = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 'key2': [2000, 2001, 2002, 2001, 2002],
                        'data': np.arange(5)})

In [69]:
right_df = pd.DataFrame(np.arange(12).reshape((6, 2)), index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                        [2001, 2000, 2000, 2000, 2001, 2002]], columns=['event1', 'event2'])

In [70]:
left_df

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [71]:
right_df

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [72]:
pd.merge(left_df, right_df, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [73]:
pd.merge(left_df, right_df, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


## Concatenating Along an Axis

In [74]:
arr = np.arange(12).reshape(4,3)

In [75]:
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [76]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11]])

In [77]:
np.concatenate([arr, arr], axis=0)

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [78]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s1

a    0
b    1
dtype: int64

In [79]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s2

c    2
d    3
e    4
dtype: int64

In [80]:
s3 = pd.Series([5, 6], index=['f', 'g'])
s3

f    5
g    6
dtype: int64

In [81]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [82]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [83]:
s4 = pd.concat([s1, s3])

In [84]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [85]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [86]:
s1

a    0
b    1
dtype: int64

In [87]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [88]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [89]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [90]:
result.unstack(level=0)

,one,two,three
a,0.0,0.0,NaN
b,1.0,1.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [91]:
result.unstack(level=1)

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [92]:
result = pd.concat([s1, s1, s3], axis=1, keys=['one', 'two', 'three'])

In [93]:
result

,one,two,three
a,0.0,0.0,NaN
b,1.0,1.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [94]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])

In [95]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [96]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])

In [97]:
df2

,three,four
a,5,6
c,7,8


In [98]:
df1.join(df2, how='inner')

,one,two,three,four
a,0,1,5,6
c,4,5,7,8


In [99]:
pd.concat([df1, df2], axis=1, keys=['level1', 'Level2'])

level1     Level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [100]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [101]:
pd.concat([df1, df2], axis=1, keys=['level1', 'Level2'], names=['uppr', 'lower'])

uppr  level1     Level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [102]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [103]:
df1

,a,b,c,d
0,-1.163568,-0.249616,1.630402,-0.018319
1,-0.890929,-1.056239,1.376497,-0.394796
2,-1.018288,0.284876,-1.732221,0.120299


In [104]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [105]:
df2

,b,d,a
0,0.790613,0.775811,-1.496129
1,-0.409216,-0.253661,-0.530343


In [106]:
pd.concat([df1, df2])

,a,b,c,d
0,-1.163568,-0.249616,1.630402,-0.018319
1,-0.890929,-1.056239,1.376497,-0.394796
2,-1.018288,0.284876,-1.732221,0.120299
0,-1.496129,0.790613,NaN,0.775811
1,-0.530343,-0.409216,NaN,-0.253661


In [107]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-1.163568,-0.249616,1.630402,-0.018319
1,-0.890929,-1.056239,1.376497,-0.394796
2,-1.018288,0.284876,-1.732221,0.120299
3,-1.496129,0.790613,NaN,0.775811
4,-0.530343,-0.409216,NaN,-0.253661


Table 8-3. concat function arguments

## Combining Data with Overlap

In [108]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])

In [109]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [110]:
b = pd.Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])

In [111]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [112]:
b[-1] = np.nan

In [113]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [114]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

Series has a combine_first method, which performs the equivalent of this operation along with pandas’s usual data alignment logic

In [115]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [116]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [117]:
b.combine_first(a)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [118]:
a.combine_first(b)

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [119]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan], 'b': [np.nan, 2., np.nan, 6.], 
                    'c': range(2, 18, 4)})

In [120]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [121]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.], 'b': [np.nan, 3., 4., 6., 8.]})

In [122]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [123]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [124]:
df2.combine_first(df1)

,a,b,c
0,5.0,NaN,2.0
1,4.0,3.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 8.3 Reshaping and Pivoting

There are a number of basic operations for rearranging tabular data. These are alternatingly referred to as *reshape* or *pivot* operations.

## Reshaping with Hierarchical Indexing

In [125]:
data = pd.DataFrame(data=np.random.randn(2,3), index=pd.Index(['Ohio', 'Colorado'], name='State'),
                    columns=pd.Index(['one', 'two', 'three'], name='Number'))

In [126]:
data

Number,one,two,three
State,,,
Ohio,-0.671293,0.767663,2.248306
Colorado,0.654767,-0.040485,0.019611


In [127]:
data.stack()

State     Number
Ohio      one      -0.671293
          two       0.767663
          three     2.248306
Colorado  one       0.654767
          two      -0.040485
          three     0.019611
dtype: float64

In [128]:
data.stack().unstack()

Number,one,two,three
State,,,
Ohio,-0.671293,0.767663,2.248306
Colorado,0.654767,-0.040485,0.019611


In [129]:
data.stack().unstack(level=0)

State,Ohio,Colorado
Number,,
one,-0.671293,0.654767
two,0.767663,-0.040485
three,2.248306,0.019611


In [130]:
data.stack().unstack(level=1)

Number,one,two,three
State,,,
Ohio,-0.671293,0.767663,2.248306
Colorado,0.654767,-0.040485,0.019611


In [131]:
data.stack().unstack('State')

State,Ohio,Colorado
Number,,
one,-0.671293,0.654767
two,0.767663,-0.040485
three,2.248306,0.019611


In [132]:
data.stack().unstack('Number')

Number,one,two,three
State,,,
Ohio,-0.671293,0.767663,2.248306
Colorado,0.654767,-0.040485,0.019611


In [133]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [134]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [135]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [136]:
s2

c    4
d    5
e    6
dtype: int64

In [137]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [138]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [139]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [140]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [141]:
df = pd.DataFrame({'left': data.stack(), 'right': data.stack() + 5}, columns=pd.Index(['left', 'right'], name='side'))

In [142]:
df

side                 left     right
State    Number                    
Ohio     one    -0.671293  4.328707
         two     0.767663  5.767663
         three   2.248306  7.248306
Colorado one     0.654767  5.654767
         two    -0.040485  4.959515
         three   0.019611  5.019611

In [143]:
df.unstack('State')

side        left               right          
State       Ohio  Colorado      Ohio  Colorado
Number                                        
one    -0.671293  0.654767  4.328707  5.654767
two     0.767663 -0.040485  5.767663  4.959515
three   2.248306  0.019611  7.248306  5.019611

In [144]:
df.unstack('State').stack('side')

State         Colorado      Ohio
Number side                     
one    left   0.654767 -0.671293
       right  5.654767  4.328707
two    left  -0.040485  0.767663
       right  4.959515  5.767663
three  left   0.019611  2.248306
       right  5.019611  7.248306

In [145]:
df.unstack('State').stack('side').unstack('Number').stack('Number')

State         Colorado      Ohio
side  Number                    
left  one     0.654767 -0.671293
      three   0.019611  2.248306
      two    -0.040485  0.767663
right one     5.654767  4.328707
      three   5.019611  7.248306
      two     4.959515  5.767663

## Pivoting “Long” to “Wide” Format

In [146]:
data = pd.read_csv('macrodata.csv')

In [147]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [148]:
data.describe()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
count,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000
mean,1983.876847,2.492611,7221.171901,4825.293103,1012.863862,663.328640,5310.540887,105.075788,667.927586,5.311773,5.884729,239.724153,3.961330,1.336502
std,14.686817,1.118563,3214.956044,2313.346192,585.102267,140.863655,2423.515977,61.278878,455.346381,2.803071,1.458574,37.390450,3.253216,2.668799
min,1959.000000,1.000000,2710.349000,1707.400000,259.764000,460.400000,1886.900000,28.980000,139.600000,0.120000,3.400000,177.146000,-8.790000,-6.790000
25%,1971.000000,1.500000,4440.103500,2874.100000,519.147500,527.959500,3276.950000,41.050000,228.650000,3.515000,4.900000,208.631000,2.270000,-0.085000
50%,1984.000000,2.000000,6559.594000,4299.900000,896.210000,662.412000,4959.400000,104.100000,540.900000,5.010000,5.700000,236.348000,3.240000,1.340000
75%,1996.500000,3.000000,9629.346500,6398.150000,1436.681500,773.049000,6977.850000,159.650000,1102.100000,6.665000,6.800000,271.721500,4.975000,2.630000
max,2009.000000,4.000000,13415.266000,9363.600000,2264.721000,1044.088000,10077.500000,218.610000,1673.900000,15.330000,10.700000,308.013000,14.620000,10.950000


In [149]:
periods = pd.PeriodIndex(year=data['year'], quarter=data['quarter'], name='date')

In [150]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [151]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [152]:
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [153]:
data = data.reindex(columns=columns)

In [154]:
data

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [155]:
data.index = periods.to_timestamp('D','end')

In [156]:
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [157]:
data.stack()

date                           item   
1959-03-31 23:59:59.999999999  realgdp     2710.349
                               infl           0.000
                               unemp          5.800
1959-06-30 23:59:59.999999999  realgdp     2778.801
                               infl           2.340
                                            ...    
2009-06-30 23:59:59.999999999  infl           3.370
                               unemp          9.200
2009-09-30 23:59:59.999999999  realgdp    12990.341
                               infl           3.560
                               unemp          9.600
Length: 609, dtype: float64

In [158]:
data.stack().reset_index()

,date,item,0
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [159]:
ldata = data.stack().reset_index().rename(columns={0: 'Value'})

In [160]:
ldata[:10]

,date,item,Value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [161]:
pivoted = ldata.pivot('date', 'item', 'Value')

In [162]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [163]:
ldata['value2'] = np.random.randn(ldata.shape[0])

In [164]:
ldata[:10]

,date,item,Value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.558396
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.241100
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.842258
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.194013
4,1959-06-30 23:59:59.999999999,infl,2.340,-1.731869
5,1959-06-30 23:59:59.999999999,unemp,5.100,-1.113994
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.957478
7,1959-09-30 23:59:59.999999999,infl,2.740,-2.021247
8,1959-09-30 23:59:59.999999999,unemp,5.300,1.720020
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.439035


In [166]:
pivoted = ldata.pivot('date', 'item')

In [168]:
pivoted[:10]

Value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.241100 -1.558396   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.731869 -0.194013   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -2.021247  0.957478   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.954236  0.439035   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.000880 -0.555911   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.531086  0.160349   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -1.867977 -0.123246   
1960-12-31 23:59:59.999999999  1.21  2802.616   6.3  0.222356 -1.004780   
1961-03-31 23:59:59.999999999 -0.40  2819.264   6.8  0.080832 -0.085869   
1961-06-30 23:59:59.999999999  1.47  2872.005   7.0  0.799954 -1.940743   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.842258  
1959-06-30 23:59:59.999999999 -1.113994  
1959-09-30 23:59:59.999999999  1.720020  
1959-12-31 23:59:59.999999999  1.666699  
1960-03-31 23:59:59.999999999 -0.621437  
1960-06-30 23:59:59.999999999  1.425413  
1960-09-30 23:59:59.999999999 -0.193254  
1960-12-31 23:59:59.999999999 -1.313306  
1961-03-31 23:59:59.999999999  0.094327  
1961-06-30 23:59:59.999999999 -1.173150

In [172]:
ldata.set_index(['date', 'item'])

Value    value2
date                          item                        
1959-03-31 23:59:59.999999999 realgdp   2710.349 -1.558396
                              infl         0.000 -0.241100
                              unemp        5.800 -0.842258
1959-06-30 23:59:59.999999999 realgdp   2778.801 -0.194013
                              infl         2.340 -1.731869
...                                          ...       ...
2009-06-30 23:59:59.999999999 infl         3.370  0.738358
                              unemp        9.200 -0.689957
2009-09-30 23:59:59.999999999 realgdp  12990.341  0.103985
                              infl         3.560  2.506206
                              unemp        9.600  1.752887

[609 rows x 2 columns]

In [175]:
ldata.set_index(['date', 'item']).unstack('item') #Equivalent to pivot

Value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8 -0.241100 -1.558396   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1 -1.731869 -0.194013   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3 -2.021247  0.957478   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6 -0.954236  0.439035   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2  0.000880 -0.555911   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0  1.597352 -1.077851   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9 -0.819511  0.930575   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1  0.292087  0.216917   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2  0.738358  0.736148   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6  2.506206  0.103985   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.842258  
1959-06-30 23:59:59.999999999 -1.113994  
1959-09-30 23:59:59.999999999  1.720020  
1959-12-31 23:59:59.999999999  1.666699  
1960-03-31 23:59:59.999999999 -0.621437  
...                                 ...  
2008-09-30 23:59:59.999999999 -0.844588  
2008-12-31 23:59:59.999999999 -0.488095  
2009-03-31 23:59:59.999999999  0.443196  
2009-06-30 23:59:59.999999999 -0.689957  
2009-09-30 23:59:59.999999999  1.752887  

[203 rows x 6 columns]

In [176]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

In [177]:
unstacked[:7]

Value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.241100 -1.558396   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.731869 -0.194013   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -2.021247  0.957478   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.954236  0.439035   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.000880 -0.555911   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.531086  0.160349   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -1.867977 -0.123246   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.842258  
1959-06-30 23:59:59.999999999 -1.113994  
1959-09-30 23:59:59.999999999  1.720020  
1959-12-31 23:59:59.999999999  1.666699  
1960-03-31 23:59:59.999999999 -0.621437  
1960-06-30 23:59:59.999999999  1.425413  
1960-09-30 23:59:59.999999999 -0.193254

## Pivoting “Wide” to “Long” Format

An inverse operation to pivot for DataFrames is pandas.melt. Rather than transforming one column into many in a new DataFrame, it merges multiple columns into
one, producing a DataFrame that is longer than the input.

In [178]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'], 'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})

In [179]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


*The 'key' column may be a group indicator, and the other columns are data values.
When using pandas.melt, we must indicate which columns (if any) are group indicators.*

In [181]:
melted = pd.melt(df, ['key'])

In [182]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [189]:
reshaped = melted.pivot('key', 'variable', 'value')

In [190]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [191]:
reshaped = melted.pivot('key', 'variable', 'value').reset_index()

In [201]:
reshaped

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [204]:
reshaped.columns.name = ''

In [205]:
reshaped

,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [206]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [208]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [209]:
pd.melt(df, value_vars=['key','A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
